## Using anoymized pcd files from bmz project using registered dataset

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import azureml.core
from azureml.core import Workspace
from azureml.core.dataset import Dataset
import json
from glob import glob
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)
import math
import os
import matplotlib.pyplot as plt
from PIL import Image

### Mount the cgm-ml repository in working directory
```
git clone https://github.com/Welthungerhilfe/cgm-ml.git
```

In [ ]:
import sys
sys.path.append('./cgm-ml')
from cgm_database import dbutils
import pandas as pd

### Store the database connection parmeters as json file

In [ ]:
db_connection_file = './ml_dbconnection.json'

### connect to azure ml workspace

In [ ]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()

### get the registered dataset from workspace

In [ ]:
dataset = Dataset.get_by_name(ws, name='anon-pcd-version5.0')
type(dataset)

### mount the dataset to compute instance

In [ ]:
mount_context = dataset.mount()
mount_context.start()  # this will mount the file streams
print(mount_context.mount_point)

### Connect to ML database

In [ ]:
ml_connector = dbutils.connect_to_main_database(db_connection_file)

### Get the column names of the the view  'artifacts_with_target'

#### artifacts_with_target contains:
- id of the artifact
- storage_path of artifact
- manual height measurement
- manual weight measurement
- scan_step
- tags

In [ ]:
columns = ml_connector.get_columns('artifacts_with_target')

In [ ]:
select_artifacts_with_targets = "select * from artifacts_with_target;"

#### Get the data as a pandas Dataframe

In [ ]:
artifacts = pd.DataFrame(ml_connector.execute(select_artifacts_with_targets, fetch_all=True), columns = columns)

In [ ]:
len(artifacts)

In [ ]:
artifacts.head()

### Filter the artifacts

### artifacts can be filter based tag and scan_step

#### Tags
- good
- bad
- delete

#### Scan Steps

- 100 - standing front scan
- 101 - standing 360 scan
- 102 - standing back scan
- 200 - lying front scan
- 201 - lying side scan
- 202 - lying back scan

### Some examples

#### Get standing front scans with tag 'good'

In [ ]:
good_scans = artifacts[(artifacts['tag']=='good')]

#### Get good standing front scans

In [ ]:
front_standing_good = artifacts[(artifacts['tag']=='good') & (artifacts['scan_step']=='100')]

### the artifacts can be accessed using this dataframe and mounted dataset

In [ ]:
artifact_list = artifacts[['storage_path', 'height', 'weight']].values.tolist()

In [ ]:
artifact_list[0]

In [ ]:
mount_path = mount_context.mount_point

In [ ]:
mount_path

In [ ]:
test_path = os.path.join(mount_path, artifact_list[0][0])    